<a href="https://colab.research.google.com/github/watex95/My-portfolio/blob/master/FORECASTING_REVENUE_WITH_Facebook_Prophet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Forecasting Hospital revenue with Facebook Prophet model
#### Predicting how much a provider can make through a period of time is, with no doubt, a powerful thing that Machine Learning models allow us to do. With a prediction of this kind we can plan new projects/expansions for the business, evaluate the impact of promotions, attract investors and much more. In order to use all these advantages, we'll be modeling a forecaster throughout this notebook, and if it has a reasonably performance, we'll be making some revenue predictions for a certain provider's revenue (sales)
#### Although we used real data, they were anonymized to increase confidentiality.

### Importing Libraries

In [9]:
# First, you need to install prophet package.
#pip install prophet

In [10]:
# Data structures and mathematical operations.
import pandas as pd
import numpy as np

# Interactive chart visualization.
import plotly.graph_objects as go
from prophet.plot import plot_plotly, plot_components_plotly

# Model and diagnostics.
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics

# Filtering warnings.
import warnings
warnings.filterwarnings('ignore')

### Loading the data and overviewing it

In [11]:
import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
import pandas as pd
data = pd.read_csv('weekly_revenue.csv')
data.head()

,Date,Visits,Revenue
0,12/28/2020,1184,10469012
1,01/04/2021,6773,64808415
2,01/11/2021,7281,72706683
3,1/18/2021,7322,80343228
4,1/25/2021,7661,77013368


### Descriptive statistics

In [12]:
data.describe(include='all')

,Date,Visits,Revenue
count,130,130.000000,1.300000e+02
unique,130,NaN,NaN
top,12/28/2020,NaN,NaN
freq,1,NaN,NaN
mean,NaN,7122.846154,7.284493e+07
std,NaN,1308.698066,1.452831e+07
min,NaN,1184.000000,1.046901e+07
25%,NaN,6416.000000,6.668491e+07
50%,NaN,7200.500000,7.585246e+07
75%,NaN,7952.000000,8.098927e+07


#### There are around 86 weeks of data. The mean weekly revenue(sales) is about 65M, with 75% (3rd quantile) of the weekly revenue being less than 81M.

In [13]:
# Firstly, we're going to covert object type to datetime type.
data.Date = pd.to_datetime(data.Date)
sales=data.drop(['Visits'],axis=1)
features = data.drop(['Revenue'],axis=1)
sales.head()


,Date,Revenue
0,2020-12-28,10469012
1,2021-01-04,64808415
2,2021-01-11,72706683
3,2021-01-18,80343228
4,2021-01-25,77013368


In [14]:
features.head()

,Date,Visits
0,2020-12-28,1184
1,2021-01-04,6773
2,2021-01-11,7281
3,2021-01-18,7322
4,2021-01-25,7661


#### To use a DataFrame as input for a Prophet model, it must have 2 columns, one named ds, for the time series, and y, for what we're predicting, which in this case is the weekly sales.

### Lets use only data upto 2023 March and forecast future sales henceforth

In [15]:
sales = sales[sales.Date<'2023-04-01']
df = sales[['Date', 'Revenue']]
df.columns = ['ds', 'y']
df.tail()

,ds,y
113,2023-02-27,61521951
114,2023-03-06,65540937
115,2023-03-13,67542318
116,2023-03-20,65591324
117,2023-03-27,73216908


#### Let's plot the data to see if we can notice some pattern.

In [36]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['ds'],
                         y=df['y'],
                         marker_color='#9597fb'))

fig.update_layout(xaxis_title="Date",
                  yaxis_title="Weekly revenue",
                  title="Weekly revenue 2021 Jan - 2023 March")
fig.show()

#### It seems that the weekly sales usually stay in a 60M - 80M range, and sporadically drops to between 25M - 50M. Initially, we don't know why, but we're going to talk about that a bit later. Now let's train our first model and see how it defines the trending and forecasts 5 weeks into the future.

### Modeling and Evaluations

In [17]:
# We're going to set some hyperparameters for all models:
# interval_width: The confidence of the trending, set to 95%;
# daily_seasonality: Indentifies daily seasonality, set to False;
# weekly_seasonality: Indentifies weekly seasonality, set to True.

m = Prophet(interval_width=.95,
            daily_seasonality=False,
            weekly_seasonality=True).fit(df)

# Forecasting 24 weeks into the future.
future = m.make_future_dataframe(periods=50, freq='W')
forecast = m.predict(future)

# Plotting.
plot_plotly(m, forecast)

DEBUG:cmdstanpy:input tempfile: /tmp/tmp7z319iic/hphwlan3.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp7z319iic/gv3j0_iy.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=34867', 'data', 'file=/tmp/tmp7z319iic/hphwlan3.json', 'init=/tmp/tmp7z319iic/gv3j0_iy.json', 'output', 'file=/tmp/tmp7z319iic/prophet_modelm0aklp0n/prophet_model-20230628130938.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
13:09:38 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
13:09:38 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


#### We can clearly see that there is a Yearly seasonality. Take a look around mid June and start of July, in all cases there was a spike, probably because of cold season. Not only rises, but periodically declines are seen, like the one after Christimas (New Year's Eve and Day). The forecasting shows this decline, which is expected to happen by the end of Christimas and throughout January. Now let's see the trend by components.

In [18]:
plot_components_plotly(m, forecast)

#### The overall trend is in decline. The spikes that we just discussed can be seen in the yearly trend chart. As it appears, Mondays are the best days for revenues, and days in the end/mid of the week are weak ones. Now we'll cross-validate the model and evaluate some performance metrics. To ease the process, we're going to define two functions. One class to suppress pystan logs will also be defined.

In [19]:
class suppress_stdout_stderr(object):

    def __init__(self):
        self.null_fds = [os.open(os.devnull, os.O_RDWR) for x in range(2)]
        self.save_fds = (os.dup(1), os.dup(2))

    def __enter__(self):
        os.dup2(self.null_fds[0], 1)
        os.dup2(self.null_fds[1], 2)

    def __exit__(self, *_):
        os.dup2(self.save_fds[0], 1)
        os.dup2(self.save_fds[1], 2)
        os.close(self.null_fds[0])
        os.close(self.null_fds[1])

In [20]:
def getCrossValidationData(m):
    with suppress_stdout_stderr():
        c_v = cross_validation(m,
                               initial='100W',   # Initially, the model will be trained in 100 weeks.
                               period='2W',      # After each model tested, we'll add 2 more weeks.
                               horizon ='2W',    # The forecasting will happen in a range of 2 weeks.
                               parallel="processes",   # To acellerate the cross-validation.
                              )
    return c_v

def getPerfomanceMetrics(m):
    return performance_metrics(getCrossValidationData(m),
                               rolling_window=1, # Generate metrics for the whole (100%) seen data.
                              )

In [21]:
getPerfomanceMetrics(m).mean()

INFO:prophet:Making 8 forecasts with cutoffs between 2022-12-05 00:00:00 and 2023-03-13 00:00:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fc980fd7850>
DEBUG:cmdstanpy:input tempfile: /tmp/tmp7z319iic/zwgk8i30.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp7z319iic/l3b24a7i.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:input tempfile: /tmp/tmp7z319iic/lv5joo6f.json
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=38272', 'data', 'file=/tmp/tmp7z319iic/zwgk8i30.json', 'init=/tmp/tmp7z319iic/l3b24a7i.json', 'output', 'file=/tmp/tmp7z319iic/prophet_model1di4wlvv/prophet_model-20230628130939.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
13:09:39 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmp7z319iic/4wenutbs.json
13:

horizon         14 days 00:00:00
mse         71860966903404.90625
rmse               8477084.81162
mae               7084076.007147
mape                    0.133898
mdape                   0.127297
smape                    0.12217
coverage                     1.0
dtype: object

#### The metrics are MSE, RMSE, MAE, MAPE, MDAPE, and SMAPE. Essencialy, if one of these metrics decreases, the other ones will also decrease, so to simplify the process of choosing the best model, the MAPE (Mean Absolute Percentage Error) metric will be picked as the value to be compared across models. This metric tells us how much the model forecasts deviate from the actual value, in mean percentage .This model, for instance, is forecasting with a mean error of 13.4%. Let's see if we can improve this value adding some known holidays¹ using Prophet syntax.

## Remove Holidays

In [22]:
new_year = pd.DataFrame({
  'holiday': 'Newyears',
  'ds': pd.to_datetime(['2021-01-01', '2022-01-01', '2023-01-01']),
  'lower_window': 0,
  'upper_window': 0,
})
easter = pd.DataFrame({
  'holiday': 'easter',
  'ds': pd.to_datetime(['2021-03-28', '2021-04-04', '2022-04-16','2022-04-17', '2023-04-06','2023-04-09']),
  'lower_window': -2,
  'upper_window': 1,
})
labor_day = pd.DataFrame({
    'holiday': "labor day",
    'ds': pd.to_datetime(['2021-05-01', '2022-05-01', '2023-05-01']),
    'lower_window': 0,
    'upper_window': 0,
})
madaraka_day = pd.DataFrame({
    'holiday': "madaraka day",
    'ds': pd.to_datetime(['2021-06-01', '2022-06-01', '2023-06-01']),
    'lower_window': 0,
    'upper_window': 0,
})
utamaduni_day = pd.DataFrame({
    'holiday': "utamaduni day",
    'ds': pd.to_datetime(['2021-10-10', '2022-10-10', '2023-10-10']),
    'lower_window': 0,
    'upper_window': 0,
})
mashujaa_day = pd.DataFrame({
    'holiday': "black friday",
    'ds': pd.to_datetime(['2021-10-20', '2022-10-20', '2023-10-20']),
    'lower_window': 0,
    'upper_window': 0,
})
jamuhuri_day = pd.DataFrame({
    'holiday': "cyber monday",
    'ds': pd.to_datetime(['2021-12-12', '2022-12-12', '2023-12-12']),
    'lower_window': 0,
    'upper_window': 0,
})

holidays = pd.concat((new_year,
                      easter,
                      labor_day,
                      madaraka_day,
                      utamaduni_day,
                      mashujaa_day,
                      jamuhuri_day
                      ))

In [23]:
m = Prophet(holidays=holidays,
            interval_width=.95,
            daily_seasonality=False)
m.add_country_holidays(country_name='KE')
with suppress_stdout_stderr():
    m.fit(df)
future = m.make_future_dataframe(periods=32, freq='W')
forecast = m.predict(future)
plot_plotly(m, forecast)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp7z319iic/z74_szex.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp7z319iic/ngpf788h.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=51601', 'data', 'file=/tmp/tmp7z319iic/z74_szex.json', 'init=/tmp/tmp7z319iic/ngpf788h.json', 'output', 'file=/tmp/tmp7z319iic/prophet_model_zbxdczz/prophet_model-20230628130940.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
13:09:40 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
13:09:40 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [24]:
getPerfomanceMetrics(m).mean()

INFO:prophet:Making 8 forecasts with cutoffs between 2022-12-05 00:00:00 and 2023-03-13 00:00:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fc9807affd0>
DEBUG:cmdstanpy:input tempfile: /tmp/tmp7z319iic/y67cuvqm.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp7z319iic/afxwf1k0.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp7z319iic/0yvc6j83.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=82208', 'data', 'file=/tmp/tmp7z319iic/y67cuvqm.json', 'init=/tmp/tmp7z319iic/0yvc6j83.json', 'output', 'file=/tmp/tmp7z319iic/prophet_modelvuxe5mn4/prophet_model-20230628130941.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
13:09:41 - cmdstanpy - INFO - Chain [1] start processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmp7z319iic/ipzzmkh_.json
INFO:cmdstanpy:Chain [1] start processing
DEB

horizon          14 days 00:00:00
mse         72705057689291.890625
rmse               8526726.082694
mae                7488952.980831
mape                     0.138679
mdape                    0.145973
smape                    0.129864
coverage                      1.0
dtype: object

#### Although now the model seems to fit the holidays better, in general, the MAPE had a slight increase, from 13.4% to 13.9%. Let's remove the holidays and add some regressors, that is, external data that can potentially influence sales, and evaluate it again.

## Add Regressors

In [25]:

# Converting it to datetime type.
features['Date'] = pd.to_datetime(features['Date'])

# Filling any null values with 0.
features.fillna(0, inplace=True)

# Merging the sales and the visits to form a new dataframe with the regressors.
df_new = features.merge(sales, on='Date')

# Renaming columns.
df_new.rename({'Date': 'ds', 'Revenue': 'y'}, axis=1, inplace=True)
features.rename({'Date': 'ds'}, axis=1, inplace=True)

# Defining regressors names iterable.
regressors = df_new.drop(['ds', 'y'], axis=1).columns

In [26]:
# Notice that the regressor is the visits
df_new.tail()

,ds,Visits,y
113,2023-02-27,6464,61521951
114,2023-03-06,6708,65540937
115,2023-03-13,6848,67542318
116,2023-03-20,6000,65591324
117,2023-03-27,6275,73216908


In [27]:
m = Prophet(interval_width=.95,
            daily_seasonality=False)

# Adding the regressors.
for regressor in regressors:
    m.add_regressor(regressor)

# Training.
with suppress_stdout_stderr():
    m.fit(df_new)

# The forecast must have the same regressors.
# In this case we're going to forecast until the end of February of the next year.
future = features[features.ds<'2023-12-30']
forecast = m.predict(future)
plot_plotly(m, forecast)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp7z319iic/dgv4w90h.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp7z319iic/991byph7.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=21123', 'data', 'file=/tmp/tmp7z319iic/dgv4w90h.json', 'init=/tmp/tmp7z319iic/991byph7.json', 'output', 'file=/tmp/tmp7z319iic/prophet_modelqds98tbo/prophet_model-20230628130942.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
13:09:42 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
13:09:42 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Lets check out the real actual sales of these forecasted sales. For example look at the forecasted sales of April 3rd week is 63 million while the actual shown below is 58 million and we find that the general deviation from the actual sales for most weeks is around 2 million to 5 million, this is around 4% to 6% margin of error.

In [35]:
data[(data.Date > '2023-04') & (data.Date < '2023-07')]

,Date,Visits,Revenue
118,2023-04-03,5825,58564293
119,2023-04-10,5936,60229283
120,2023-04-17,6200,65596856
121,2023-04-24,6767,66956635
122,2023-05-01,5038,50741606
123,2023-05-08,6857,74354321
124,2023-05-15,6912,73621065
125,2023-05-22,7197,76902049
126,2023-05-29,6123,63830221
127,2023-06-05,6990,76498785


### Lets check out the final performance of the model.

In [29]:
getPerfomanceMetrics(m).mean()

INFO:prophet:Making 8 forecasts with cutoffs between 2022-12-05 00:00:00 and 2023-03-13 00:00:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fc98423b0d0>
DEBUG:cmdstanpy:input tempfile: /tmp/tmp7z319iic/zozdtoah.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp7z319iic/um23fjn8.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp7z319iic/c8759me_.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=20996', 'data', 'file=/tmp/tmp7z319iic/zozdtoah.json', 'init=/tmp/tmp7z319iic/c8759me_.json', 'output', 'file=/tmp/tmp7z319iic/prophet_modeln2xqhr49/prophet_model-20230628130943.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
13:09:43 - cmdstanpy - INFO - Chain [1] start processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmp7z319iic/c8ustba6.json
INFO:cmdstanpy:Chain [1] start processing
DEB

horizon          14 days 00:00:00
mse         38959373881766.328125
rmse               6241744.458224
mae                5273858.150773
mape                     0.092503
mdape                    0.089347
smape                    0.088485
coverage                   0.9375
dtype: object

#### MAPE had  a significant change, its final value is around 9.3%, which means that if the model predicts a value for a time series in the future, the forecasts will deviate from the actual values, on average, by 9.3%.

### Conclusion
#### After modeling three different forecasters, we found out that the best one performs a (mean absolute percentage error) MAPE of 9.3%, which is quite a good value, showing that the model can be used to forecast sales if high precision is desired. The explanation for this is because the provider has other factors that influence revenue/sales  that the model will become better when we add more regressors/features.

##### ² https://www.statology.org/what-is-a-good-mape/